In [1]:
from demoparser2 import DemoParser
import pandas as pd
from nade_efficacy_pipeline_with_debugging import process_multiple_demos

In [2]:
demo_list = [
    'g2-vs-faze-m1-ancient-p1.dem',
    'g2-vs-faze-m1-ancient-p2.dem',
    'g2-vs-faze-m2-inferno-p1.dem',
    'g2-vs-faze-m2-inferno-p2.dem',
    'g2-vs-faze-m3-nuke.dem'
]

result = process_multiple_demos(demo_list)

Player names extracted: ['karrigan' 'Snax' 'broky' 'EliGE' 'm0NESY' 'rain' 'HeavyGoD' 'frozen'
 'huNter-' 'malbsMd' None]
HE grenade detonation counts: user_name
malbsMd     7
huNter-     6
HeavyGoD    4
frozen      4
Snax        3
broky       2
rain        2
karrigan    1
m0NESY      1
EliGE       1
Name: count, dtype: int64
Player names extracted: ['HeavyGoD' 'frozen' 'EliGE' 'karrigan' 'broky' 'rain' 'm0NESY' 'huNter-'
 'malbsMd' 'Snax' None]
HE grenade detonation counts: user_name
EliGE       15
rain        14
frozen      12
malbsMd     12
karrigan    10
Snax         9
huNter-      9
HeavyGoD     7
m0NESY       5
broky        4
Name: count, dtype: int64
Player names extracted: ['rain' 'EliGE' 'm0NESY' 'Snax' 'broky' 'frozen' 'karrigan' 'huNter-'
 'HeavyGoD' 'malbsMd' None]
HE grenade detonation counts: user_name
frozen      9
karrigan    9
malbsMd     6
EliGE       5
rain        4
broky       3
Snax        3
HeavyGoD    3
m0NESY      1
huNter-     1
Name: count, dtype: int64
Player

In [3]:
def nade_cleaner(dfs):
    results = {}
    counter = 1
    for _, df in dfs.items():
       cleaned_name = f"demo_cleaned{counter}"
       results[cleaned_name] = df.iloc[:, :3].copy()
       counter+=1
    return results

naked_nades = nade_cleaner(result)

In [4]:
naked_nades['demo_cleaned1']

,user_name,count,dmg_health
0,karrigan,1,0.0
1,Snax,3,173.0
2,broky,2,51.0
3,EliGE,1,31.0
4,m0NESY,1,0.0
5,rain,2,0.0
6,HeavyGoD,4,24.0
7,frozen,4,134.0
8,huNter-,6,46.0
9,malbsMd,7,3.0


In [5]:
'''Merges multiple demo dataframes, summing the count and dmg_health columns for each player across all dataframes.
  It calculates the average damage per grenade (avg_dmg) and returns the aggregated dataframe, handling missing values with 0.'''

def merge_and_sum(naked_nades):
    # Start with the first DataFrame in the dictionary
    merged_df = list(naked_nades.values())[0]
    
    # Iterate over the remaining DataFrames and merge them
    for df in list(naked_nades.values())[1:]:
        merged_df = pd.merge(merged_df, df, on='user_name', how='outer', suffixes=('', '_y'))
        
        # Sum the 'count_x' and 'he_damage' columns (handle '_y' suffix)
        merged_df['count'] += merged_df['count_y'].fillna(0)
        merged_df['dmg_health'] += merged_df['dmg_health_y'].fillna(0)
        
        # Drop the extra '_y' columns
        merged_df.drop(columns=[col for col in merged_df if col.endswith('_y')], inplace=True)
        merged_df['avg_dmg'] = (merged_df['dmg_health'] / merged_df['count']).round(1)
    
    return merged_df

In [7]:
final_df = merge_and_sum(naked_nades)
final_df.sort_values(by='avg_dmg',ascending=False)

,user_name,count,dmg_health,avg_dmg
7,frozen,46,739.0,16.1
1,Snax,28,369.0,13.2
8,huNter-,35,443.0,12.7
9,malbsMd,42,414.0,9.9
4,m0NESY,23,225.0,9.8
3,EliGE,33,258.0,7.8
2,broky,17,122.0,7.2
6,HeavyGoD,30,194.0,6.5
0,karrigan,34,154.0,4.5
5,rain,29,109.0,3.8
